## This is an example of model training from Jupyter notebook

#### import packages and load dataset

In [ ]:
from pathlib import Path

from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
import pandas as pd

from chemprop import data, featurizers, models, nn


input_path = "./data.csv" # path to the input CSV fil
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'MOL_smiles' # name of the column containing SMILES strings
target_columns = ['LOG_HLM_CLint', 
                  'LOG_RLM_CLint',
                  'LOG_MDR1-MDCK_ER',
                  'LOG_HPPB',
                  'LOG_RPPB',
                   'LOG_SOLUBILITY'] # list of names of the columns containing targets
df_input = pd.read_csv(input_path)
df_input.head()


#### Data preparation
- Get smiles and target data from CSV file
- combine structure and target dataset

In [ ]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values


In [ ]:
smis[:5]

In [ ]:
ys[:5]

In [ ]:
# https://chemprop.readthedocs.io/en/latest/tutorial/python/data/datapoints.html
# Create a list of MoleculeDatapoint objects from SMILES strings and target values
all_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

In [ ]:
list(data.SplitType.keys())

In [ ]:
# Example of a random split
mols = [d.mol for d in all_data]  # RDkit Mol objects are use for structure based splits
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))  # unpack the tuple into three separate lists
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

In [ ]:
# https://chemprop.readthedocs.io/en/latest/autoapi/chemprop/featurizers/index.html#module-chemprop.featurizers
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_dset = data.MoleculeDataset(train_data[0], featurizer)
scaler = train_dset.normalize_targets()

val_dset = data.MoleculeDataset(val_data[0], featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data[0], featurizer)

In [ ]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

In [ ]:
mp = nn.BondMessagePassing()
#agg = nn.MeanAggregation()
agg = nn.SumAggregation()
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
ffn = nn.RegressionFFN(n_tasks=len(target_columns), output_transform=output_transform)
batch_norm = True

In [ ]:
print(nn.metrics.MetricRegistry)

In [ ]:
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]

In [ ]:
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)
mpnn

In [ ]:
# Configure model checkpointing
checkpointing = ModelCheckpoint(
    "note_checkpoints",  # Directory where model checkpoints will be saved
    "best-{epoch}-{val_loss:.2f}",  # Filename format for checkpoints, including epoch and validation loss
    "val_loss",  # Metric used to select the best checkpoint (based on validation loss)
    mode="min",  # Save the checkpoint with the lowest validation loss (minimization objective)
    save_last=True,  # Always save the most recent checkpoint, even if it's not the best
)

In [ ]:
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=20, # number of epochs to train for
    callbacks=[checkpointing], # Use the configured checkpoint callback
)

In [ ]:
trainer.fit(mpnn, train_loader, val_loader)

In [ ]:
results = trainer.test(dataloaders=test_loader)